# 70. データの入手・整形

まずは必要となるpositive/negativeのデータセットをインストールします。

In [ ]:
!bash ../shell/setup_ch8_dataset.sh

ファイルが存在することを確認します。

In [ ]:
!ls ../data/raw/rt-polaritydata/

ファイルの中身を見てみましょう。まずは rt-polarity.pos (ポジティブな文のデータセット) です

In [ ]:
!head -n 3 ../data/raw/rt-polaritydata/rt-polarity.pos

次は rt-polarity.neg (ネガティブな文のデータセット) です

In [ ]:
!head -n 3 ../data/raw/rt-polaritydata/rt-polarity.neg

## 70-1 rt-polarity.posの各行の先頭に"+1 "という文字列を追加する

さて1つ目のタスクです。

エディタを開いてPythonコードを…とやりたくなるかもしれませんが、このくらいの加工ならターミナル上でワンライナーで書けるようになりましょう。

Pythonのコードをいちいち書くよりも簡単に高速に書くことができるため、作業の効率化に繋がります。

求められていることは、 1つのファイル内の **各行で** 同様の処理を行うことです。このような場合は、 **awk** を 使います。

awkをインストールしましょう
環境によりインストール方法が異なるので、各自調べてください。以下のコマンドが成功すればOKです

In [ ]:
!echo '' | awk '{print "OK"}'

以下のワンライナーで、やりたいことを実現できます。

In [ ]:
!cat ../data/raw/rt-polaritydata/rt-polarity.pos | awk '{print "+1 "$0}' > ../data/preprocessing/rt-polarity.pos

解説です。awkの基本構文は、

```sh
awk '{ (各行でやりたいこと) }'
```

です。Pythonで表現するとしたら、

```py
import sys

for line in sys.stdin:
    line = line.rstrip()
    (各行でやりたいこと)
```

ですね。

今回のタスクでは、各行に対して先頭に `"+1 "` という文字列を追加して出力するということをしたいです。

出力のために使う関数が `print()` です。

`$0` は 「今見ている行 (文字列) 」です。

文字列は横に並べると結合できるので、 `"+1 "$0` は「+1」「(スペース)」「今見ている行の文字列」を結合した文字列を意味します。

この文字列を `print()` して、やりたいことが実現できるわけです。

【ひとこと】

インストールした圧縮ファイル (compression)、展開したファイル (raw)、システムへの入力となるファイル (preprocessing)のように、別のディレクトリに保存する習慣を付けましょう。

元のファイルを上書きしてしまうと、別のやり方を試そうと思った時にもう一度インストールするところから始めなければいけません。

## 70-2 rt-polarity.posの各行の先頭に"-1 "という文字列を追加する

先ほどのやり方を応用して awk で書いてみましょう。

以下のセルにシェルコマンドを書いてください。出力先は ../data/preprocessing/rt-polarity.neg です。

In [ ]:
!echo ''

確認します。"OK" が表示されれば成功しています。

In [ ]:
! [[ -e ../data/preprocessing/rt-polarity.neg ]] && [[ $(grep -c -v '^-1 [^ ]' ../data/preprocessing/rt-polarity.neg) -ne 0 ]] && [[ $(cat ../data/raw/rt-polaritydata/rt-polarity.neg | wc -l) -eq $(cat ../data/preprocessing/rt-polarity.neg | wc -l) ]] && echo "OK"

## 70-3 posとnegのファイルを結合し、行をランダムに並び替える

これもワンライナーですぐにできます。

2つのファイルを **結合** するためには `cat` コマンドを使います。

あるファイルの中身を **ランダムに並び替える** には、 `sort -R` コマンドを使います。

これまでにも使っていましたが、シェルの機能である **パイプ** を使うと、上の2つの処理を続けて行うことができます。

In [ ]:
# 失敗します
!cat ../data/preprocessing/rt-polarity.pos ../data/preprocessing/rt-polarity.neg | sort -R

In [ ]:
# おまじないを付けて実行
!cat ../data/preprocessing/rt-polarity.pos ../data/preprocessing/rt-polarity.neg | LC_ALL=C sort -R > ../data/preprocessing/sentiment.txt

In [ ]:
# 確認
!head -n 5 ../data/preprocessing/sentiment.txt

In [ ]:
# 正例の数の確認
!grep "^+1"  ../data/preprocessing/sentiment.txt | wc -l

In [ ]:
# 負例の数の確認
!grep "^-1"  ../data/preprocessing/sentiment.txt | wc -l

# 71. ストップワード

※ 時間の関係で省略

# 72. 素性抽出
極性分析に有用そうな素性を各自で設計し,学習データから素性を抽出せよ.素性としては,レビューからストップワードを除去し,各単語をステミング処理したものが最低限のベースラインとなるであろう.

実際のタスクでは言語的な特徴をいかに捉えて素性として表現するかがポイントなのですが、今回はscikit-learnの関数でサクッと実現しましょう。

単語の回数でベクトル化する CountVectorizer
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=vectorizer#sklearn.feature_extraction.text.CountVectorizer

In [ ]:
# 別途書いておいた関数を呼べるように、パス追加
import sys
import os

# 書き換えてください
project_home = '~/project/coworker-jp/nlp-knock-handson'
if not os.path.exists(os.path.expanduser(project_home)):
    raise RuntimeError('git cloneで作成したディレクトリのパスで書き換えてください')

sys.path.append(project_home + '/jp/co/coworker/nlp_knock_handson/ch8/')

%cd $project_home
%pwd

from jp.co.coworker.nlp_knock_handson.ch8.config import Config
from jp.co.coworker.nlp_knock_handson.ch8.preprocessor import PreProcessor
preprocessor = PreProcessor()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_curve

import matplotlib.pyplot as plt

import nltk
nltk.download('popular', quiet=True)

In [ ]:
# データの読み込み
texts, Y = PreProcessor.load_dataset(Config.get_dataset_path())

# 単語の回数でベクトル化 (最も簡単な例)
# decode_error='ignore' は、入力の単語にutf-8外の文字があった場合の設定。おまじない
vectorizer = CountVectorizer(decode_error='ignore')
X_simple = vectorizer.fit_transform(texts)

feature_words = vectorizer.get_feature_names()
print(feature_words[100:200])


In [ ]:
# 英語のストップワード除去
stop_words = preprocessor.get_stop_words()
vectorizer = CountVectorizer(decode_error='ignore', stop_words=stop_words)
X = vectorizer.fit_transform(texts)

In [ ]:
# ステミング + プリセットで登録されているstop_wordsを使う
stemmer = nltk.stem.lancaster.LancasterStemmer()
print(stemmer.stem("studied"))
print(stemmer.stem("cats"))

# 単語分割
print(nltk.word_tokenize("Hello, have a nice day."))

def stem_analyzer(text):
    tokens = nltk.word_tokenize(text)
    return [stemmer.stem(token)  for token in tokens]

In [ ]:
vectorizer = CountVectorizer(decode_error='ignore', stop_words='english', analyzer=stem_analyzer)
X = vectorizer.fit_transform(texts)

In [ ]:
feature_words = vectorizer.get_feature_names()
print(feature_words[10:90])

# 73. 学習
72で抽出した素性を用いて,ロジスティック回帰モデルを学習せよ.

In [ ]:
clf = LogisticRegression().fit(X_simple, Y)

# 74. 予測
73で学習したロジスティック回帰モデルを用い,与えられた文の極性ラベル(正例なら"+1",負例なら"-1")と,その予測確率を計算するプログラムを実装せよ.

In [ ]:
Y_pred = clf.predict(X_simple)

prob_list = clf.predict_proba(X_simple)
Y_pred_prob = [neg_pos[1] for neg_pos in prob_list]

Y_pred_prob[0:10]

# 75. 素性の重み

In [ ]:
# clf.coef_ に 重みの情報が入っている
# ( クラス数 *  素性数 ) の2次元配列
weight_list = clf.coef_[0]
print(weight_list)

# 素性がどの単語を表しているかは、get_feature_names()で分かる
feature_words = vectorizer.get_feature_names()
print(feature_words[100:200])

In [ ]:
weight_pairs = zip(weight_list, feature_words)
sorted_weight_pairs = sorted(weight_pairs, key=lambda p: p[0])

worst10 = sorted_weight_pairs[:10]
print(worst10)

top10 = sorted_weight_pairs[-10:]
print(top10)


# 76. ラベル付け

In [ ]:
# 整形するだけ
a = zip(Y, Y_pred, Y_pred_prob, texts)

for pairs in list(a)[:10]:
  print(pairs)

# 77. 正解率の計測

In [ ]:
# sklearnに関数がある
precision = precision_score(Y, Y_pred)
recall = recall_score(Y, Y_pred)
f1 = f1_score(Y, Y_pred)

print(precision)
print(recall)
print(f1)

# 78. 5分割交差検定

In [ ]:
# https://scikit-learn.org/stable/modules/cross_validation.html

scorers_dict = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
}

scores = cross_validate(clf, X, Y, scoring=scorers_dict)

scores

# 79. 適合率-再現率グラフの描画

In [ ]:
precision, recall, threshold = precision_recall_curve(Y, Y_pred_prob)

plt.plot(precision, recall)
plt.xlabel('Precision')
plt.ylabel('Recall')

plt.show()